<a href="https://colab.research.google.com/github/pratham11kaushik/twitter_sentiment_analysis_using_ML.ipynb/blob/main/new_data_tweetwe_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# importing the dependencies

In [ ]:

import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# printing the stopwords in english
print (stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# data processing

In [ ]:
import pandas as pd

# loading the data from csv file to pandas dataframe
twitter_data = pd.read_csv('/content/twitter_training.csv', encoding= 'ISO-8859-1')

In [ ]:
# CHECKING THE NO. OF rows and columns
twitter_data.shape

(68431, 4)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# printing the first 5 rows of the dataframe
twitter_data.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [ ]:
# naming the  colums and reading the dataset again

column_names = ['id','user platform','target','text']
twitter_data = pd.read_csv('/content/twitter_training.csv', names=column_names, encoding= 'ISO-8859-1')


# CHECKING THE NO. OF rows and columns
twitter_data.shape

(74682, 4)

In [ ]:

# printing the first 5 rows of dataframe
twitter_data.head()

,id,user platform,target,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [ ]:
# counting the no. of missing values in the dataset
twitter_data.isnull().sum()

id                 0
user platform      0
target             0
text             686
dtype: int64

In [25]:
twitter_data.fillna(0, inplace=True)

In [119]:
# counting the no. of missing values in the dataset  # mission completed
twitter_data.isnull().sum()

id                 0
user platform      0
target             0
text               0
stemmed_content    0
new_target         0
dtype: int64

In [ ]:
# checking the distribution of target column
twitter_data['target'].value_counts()

target
Negative      22542
Positive      20832
Neutral       18318
Irrelevant    12990
Name: count, dtype: int64

# convert the target "4" to "1"

In [118]:
from threading import Condition
twitter_data.replace({'target':{'positive':1}}, inplace=True)

Condition_mapping = {
    'Negative' : 0,
    'Positive' : 1,
    'Neutral'  : 0.5,
    'Irrelevant':-1
}

In [117]:
print(twitter_data['target'])


0        Positive
1        Positive
2        Positive
3        Positive
4        Positive
           ...   
74677    Positive
74678    Positive
74679    Positive
74680    Positive
74681    Positive
Name: target, Length: 74682, dtype: object


In [116]:
 # checking the distribution of target column
twitter_data['target'].value_counts()


target
Negative      22542
Positive      20832
Neutral       18318
Irrelevant    12990
Name: count, dtype: int64

**0--------> Negative tweet
1--------> positive tweet
Stemming
steming is the process of reduceing a word to its roots word, example -actor, actress, acting = act**

In [41]:
#from nltk.stem import PorterStemmer
#import re
#twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)# 50 minutes to complete thus execution


In [134]:
import nltk
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

def stemming(content):
    """
    Stems a piece of text content efficiently.

    Args:
        content (str): The text content to be stemmed.

    Returns:
        str: The stemmed text content.

    Raises:
        TypeError: If the input content is not a string.
    """

    if not isinstance(content, str):
        raise TypeError("Input content must be a string.")

    # Preprocess: lowercase, remove non-alphanumeric characters
    stemmed_content = re.sub(r"[^a-zA-Z0-9]", " ", content.lower())

    # Tokenize, filter stopwords, stem using PorterStemmer
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english'))  # Convert to a set for faster membership check
    stemmed_tokens = [stemmer.stem(word) for word in stemmed_content.split() if word not in stop_words]

    # Join stemmed tokens back into a string (optional, depending on use case)
    return ' '.join(stemmed_tokens)

# Example usage
nltk.download('punkt', quiet=True)  # Download NLTK punkt tokenizer (if not already downloaded)
nltk.download('stopwords', quiet=True)  # Download NLTK stopwords (if not already downloaded)

# Assuming you have a DataFrame called 'twitter_data' with a column named 'text'
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)


TypeError: Input content must be a string.

In [101]:
twitter_data.head()

,id,user platform,target,text,stemmed_content,new_target
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,g e n g n b r e r l n n w l l u r e r u l l,1.0
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,c n g h e b r e r n w l l k l l u l l,1.0
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,g e n g n b r e r l n n w l l k l l u l l,1.0
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,c n g n b r e r l n n w l l u r e r u l l,1.0
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,g e n g n b r e r l n 2 n w l l u r e r u e l l,1.0


In [102]:
print(twitter_data['stemmed_content'])

0              g e n g n b r e r l n n w l l u r e r u l l
1                    c n g h e b r e r n w l l k l l u l l
2                g e n g n b r e r l n n w l l k l l u l l
3                c n g n b r e r l n n w l l u r e r u l l
4          g e n g n b r e r l n 2 n w l l u r e r u e l l
                               ...                        
74677    j u r e l z e h h e w n w p r n f c l k e 6 e ...
74678    j u r e l z e h c w n w p r n 6 e r b e h n n ...
74679    j u r e l z e h e w n w p r n f c n w 6 e r b ...
74680    j u r e l z e b e w e e n h e w n w p r n f c ...
74681    j u l k e h e w n w p r n f c l k e 6 e r b e ...
Name: stemmed_content, Length: 74682, dtype: object


In [103]:
twitter_data['new_target'] = twitter_data['target'].map(Condition_mapping)
Condition_mapping = {
    'Negative' : 0,
    'Positive' : 1,
    'Neutral'  : 0.5,
    'Irrelevant':-1
}
train_X = twitter_data['stemmed_content'].values
train_Y = twitter_data['new_target'].values

In [104]:
print(train_X)

['g e n g n b r e r l n n w l l u r e r u l l'
 'c n g h e b r e r n w l l k l l u l l'
 'g e n g n b r e r l n n w l l k l l u l l' ...
 'j u r e l z e h e w n w p r n f c n w 6 e r b e h n n n v r v e r n h v e n e h w h e n n c e'
 'j u r e l z e b e w e e n h e w n w p r n f c l k e b e n g 6 e r b e h n n n v r v e r n c r h v e n f u c k n g e h w e v e r n n c e'
 'j u l k e h e w n w p r n f c l k e 6 e r b e h n n r v e r u h v e n e h w n n c e']


In [74]:
Y_train = 'Y'
Y_train = np.array(train_Y)

In [75]:
print(Y_train)

[1. 1. 1. ... 1. 1. 1.]


#  test data set

In [58]:
import pandas as pd

# loading the data from csv file to pandas dataframe
test_data = pd.read_csv('//content/twitter_validation.csv', encoding= 'ISO-8859-1')


# CHECKING THE NO. OF rows and columns
test_data.shape


(999, 4)

In [59]:
# naming the  colums and reading the dataset again

column_names = ['id','user platform','target','text']
test_data = pd.read_csv('/content/twitter_validation.csv', names=column_names, encoding= 'ISO-8859-1')


# CHECKING THE NO. OF rows and columns
test_data.shape

(1000, 4)

In [60]:
# printing the first 5 rows of dataframe
test_data.head()

,id,user platform,target,text
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [61]:
# counting the no. of missing values in the dataset
test_data.isnull().sum()


id               0
user platform    0
target           0
text             0
dtype: int64

In [62]:
# checking the distribution of target column
test_data['target'].value_counts()

target
Neutral       285
Positive      277
Negative      266
Irrelevant    172
Name: count, dtype: int64

In [63]:
print(test_data['target'])


0      Irrelevant
1         Neutral
2        Negative
3        Negative
4         Neutral
          ...    
995    Irrelevant
996    Irrelevant
997      Positive
998      Positive
999       Neutral
Name: target, Length: 1000, dtype: object


In [64]:
#def stemming(content):

  # Check if content is a string
  #if not isinstance(content, str):
   # content = str(content)  # Convert to string if necessary

  #stemmed_content = re.sub(r"[^a-zA-Z0-9]", " ", content.lower())
  #stemmed_content = stemmed_content.split()
  #stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  #stemmed_content = ''.join(stemmed_content)

  #return stemmed_content

In [66]:
#import nltk
#import re
#test_data['stemmed_content2'] = test_data['text'].apply(stemming)# 50 minutes to complete thus execution


In [67]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

def stemming(content):
    """
    Stems a piece of text content efficiently.

    Args:
        content (str): The text content to be stemmed.

    Returns:
        str: The stemmed text content.

    Raises:
        TypeError: If the input content is not a string.
    """

    if not isinstance(content, str):
        raise TypeError("Input content must be a string.")

    # Preprocess: lowercase, remove non-alphanumeric characters
    stemmed_content = re.sub(r"[^a-zA-Z0-9]", " ", content.lower())

    # Tokenize, filter stopwords, stem using PorterStemmer
    stemmer = PorterStemmer()
    stop_words = stopwords.words('english')
    stemmed_tokens = [stemmer.stem(word) for word in stemmed_content.split() if word not in stop_words]

    # Join stemmed tokens back into a string (optional, depending on use case)
    return ' '.join(stemmed_tokens)

# Example usage
nltk.download('punkt', quiet=True)  # Download NLTK punkt tokenizer (if not already downloaded)
nltk.download('stopwords', quiet=True)  # Download NLTK stopwords (if not already downloaded)

test_data['stemmed_content2'] = test_data['text'].apply(stemming)


In [68]:
print(test_data['stemmed_content2'])

0      mention facebook struggl motiv go run day tran...
1      bbc news amazon boss jeff bezo reject claim co...
2      microsoft pay word function poorli samsungu ch...
3           csgo matchmak full closet hack truli aw game
4      presid slap american face realli commit unlaw ...
                             ...                        
995    toronto art cultur capit canada wonder want st...
996    actual good move tot bring viewer one peopl go...
997    today suck time drink wine n play borderland s...
998            bought fraction microsoft today small win
999    johnson johnson stop sell talc babi powder u c...
Name: stemmed_content2, Length: 1000, dtype: object


In [69]:
print(test_data['target'])

0      Irrelevant
1         Neutral
2        Negative
3        Negative
4         Neutral
          ...    
995    Irrelevant
996    Irrelevant
997      Positive
998      Positive
999       Neutral
Name: target, Length: 1000, dtype: object


In [70]:
test_data['new_target1'] = test_data['target'].map(Condition_mapping)
Condition_mapping = {
    'Negative' : 0,
    'Positive' : 1,
    'Neutral'  : 0.5,
    'Irrelevant':-1
}


In [71]:
#seprating the data and label
test_Z = test_data['stemmed_content2'].values
test_C = test_data['new_target1'].values


print(test_Z)

['mention facebook struggl motiv go run day translat tom great aunti hayley get bed told grandma think lazi terribl person'
 'bbc news amazon boss jeff bezo reject claim compani act like drug dealer bbc co uk news av busin'
 'microsoft pay word function poorli samsungu chromebook'
 'csgo matchmak full closet hack truli aw game'
 'presid slap american face realli commit unlaw act acquitt discov googl vanityfair com news 2020 02'
 'hi eahelp madelein mccann cellar past 13 year littl sneaki thing escap whilst load fifa point took card use paypal account work help resolv pleas'
 'thank eamaddennfl new te austin hooper orang brown brown austinhooper18 pic twitter com grg4xzfkon'
 'rocket leagu sea thiev rainbow six sieg love play three stream best stream twitch rocketleagu seaofthiev rainbowsixsieg follow'
 'ass still knee deep assassin creed odyssey way anytim soon lmao'
 'fix jesu pleas fix world go playstat askplayst playstationsup treyarch callofduti neg 345 silver wolf error code pic t

In [72]:
print(test_C)

[-1.   0.5  0.   0.   0.5  0.   1.   1.   1.   0.   1.   1.   0.   0.5
  0.   1.   1.   0.   1.   0.   0.   0.5 -1.   0.   0.5  0.5  0.  -1.
 -1.   0.   1.   1.   0.   1.   0.   0.5  0.5 -1.   1.   0.5  1.   0.5
  0.5  0.5  1.   0.5  0.   0.   0.   0.5  1.   0.   0.   1.   1.   1.
  1.   1.   0.  -1.   0.   1.   1.  -1.   0.   0.5  0.  -1.   0.5  0.
  1.   0.   0.   1.   1.  -1.   1.  -1.   0.5  0.5  0.5  1.   1.   0.5
  1.   0.5  0.  -1.   0.   0.5  0.5  0.   1.  -1.  -1.   0.   0.   0.
  0.5  1.   0.5  0.   1.   1.   0.5  1.   0.5  1.   0.   0.5  0.5  0.5
  0.   0.5  0.   0.5  0.5  1.   1.   0.5  0.   0.   1.   0.   0.5  0.
  1.   0.5  0.   0.5 -1.   1.   0.5  1.   1.  -1.   0.5  0.5 -1.  -1.
 -1.   0.5  0.5 -1.  -1.  -1.   1.   0.5  1.  -1.   1.   0.   0.5  0.5
  0.5 -1.   0.5  0.   0.5  1.   0.   0.5  0.  -1.  -1.  -1.   0.5  0.
  0.   0.   1.   1.   1.   0.5  0.5  1.  -1.   0.5  0.5  0.5  1.   0.5
  0.   0.   0.5  1.   1.  -1.  -1.   0.5  1.   1.   0.5 -1.   0.5  0.
  0.   0.   0.

# Spliting the data into training data and test data

In [79]:
from sklearn.model_selection import train_test_split
#seprating the data and label
#X = twitter_data['stemmed_content'].values
#Y = twitter_data['new_target'].values
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)



#print(X.shape, X_train.shape, X_test.shape)

print (train_X)

['  g e n g   n   b r e r l n   n     w l l   u r e r   u   l l    '
 '    c n g     h e   b r e r   n     w l l   k l l   u   l l  '
 '  g e n g   n   b r e r l n   n     w l l   k l l   u   l l  ' ...
 'j u   r e l z e   h e   w n w   p r n   f     c     n w   6   e r   b e h n   n   n v   r v e r   n     h v e   n   e   h w   h e   n         n c e'
 'j u   r e l z e   b e w e e n   h e   w n w   p r n   f     c     l k e   b e n g   6   e r   b e h n   n   n v   r v e r   n   c r     h v e   n   f u c k n g   e   h w     e v e r   n             n c e'
 'j u   l k e   h e   w n w   p r n   f     c     l k e   6   e r   b e h n   n     r v e r     u   h v e   n   e   h w     n         n c e']


In [80]:
print(test_Z)


['mention facebook struggl motiv go run day translat tom great aunti hayley get bed told grandma think lazi terribl person'
 'bbc news amazon boss jeff bezo reject claim compani act like drug dealer bbc co uk news av busin'
 'microsoft pay word function poorli samsungu chromebook'
 'csgo matchmak full closet hack truli aw game'
 'presid slap american face realli commit unlaw act acquitt discov googl vanityfair com news 2020 02'
 'hi eahelp madelein mccann cellar past 13 year littl sneaki thing escap whilst load fifa point took card use paypal account work help resolv pleas'
 'thank eamaddennfl new te austin hooper orang brown brown austinhooper18 pic twitter com grg4xzfkon'
 'rocket leagu sea thiev rainbow six sieg love play three stream best stream twitch rocketleagu seaofthiev rainbowsixsieg follow'
 'ass still knee deep assassin creed odyssey way anytim soon lmao'
 'fix jesu pleas fix world go playstat askplayst playstationsup treyarch callofduti neg 345 silver wolf error code pic t

In [107]:
from sklearn.feature_extraction.text import TfidfVectorizer# converting the textual data to numerical data

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(train_X)
Z_test = vectorizer.transform(test_Z)



ValueError: empty vocabulary; perhaps the documents only contain stop words

In [86]:
print (X_train)
print (Z_test)

NameError: name 'Z_test' is not defined

# training the machine learning model
logestic regression

In [135]:
from sklearn.linear_model import LogisticRegression  # Import LogisticRegression
from sklearn.model_selection import train_test_split

model = LogisticRegression(max_iter=1000)  # Set max_iter to a valid integer value

# Reshape Y_train to have the same number of samples as X_train
Y_train = Y_train.reshape(-1, 1)  # Fix the shape of Y_train

In [137]:
model.fit(train_X, train_Y)

ValueError: could not convert string to float: 'g e n g n b r e r l n n w l l u r e r u l l'

# Model evaluation

Accuracy score

In [ ]:
from sklearn.metrics import accuracy_score
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)


print ('accuracy score on the training data: ', training_data_accuracy)

In [ ]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)


print ('accuracy score on the training data: ', test_data_accuracy)

# using the saved model for future prediction

In [ ]:
# loading the savedmodel
loaded_model = pickle.load(open('/content/trained_model.sav','rb'))


x_new = X_test[200]
print(Y_test[200])

prediction = model.predict(X_test)
print(prediction)

if (prediction[0] == 0) :
  print('Negative tweet')

else:
     print('positive tweet')


In [ ]:
x_new = X_test[528]
print(Y_test[528])

prediction = model.predict(X_test)
print(prediction)

if (prediction[0] == 0) :
  print('Negative tweet')

else:
     print('positive tweet')
